In [3]:
import pandas as pd

In [35]:
df = pd.read_csv('../ml-latest-small/movies.csv')
df_notas = pd.read_csv('../ml-latest-small/ratings.csv')

In [36]:
df_notas.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [46]:
total_score = df_notas['movieId'].value_counts()
total_score = total_score.reset_index()
total_score.columns = ['movieId', 'total_score']

In [47]:
df.merge(total_score, 'inner', 'movieId')

,movieId,title,genres,total_score
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,2,Jumanji (1995),Adventure|Children|Fantasy,110
2,3,Grumpier Old Men (1995),Comedy|Romance,52
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7
4,5,Father of the Bride Part II (1995),Comedy,49
...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1
9721,193585,Flint (2017),Drama,1
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,1
